In [5]:
import pandas as pd

df = pd.read_parquet("/home/hieuvd/lvdthieu/CodeGen/java_data/data/compile_info_codellama_v1.parquet", "fastparquet")
# df.to_parquet("/home/hieuvd/lvdthieu/CodeGen/java_data/data/deepseek.parquet", "fastparquet")
len(df[df["compile_info_filled_file_baseline_output"] == "<COMPILED_SUCCESSFULLY>"]) / len(df)

0.9578396252411132

In [9]:
import pandas as pd

df = pd.read_parquet("/home/hieuvd/lvdthieu/CodeGen/java_data/data/compile_info_deepseek_v1.parquet", "fastparquet")
# df.to_parquet("/home/hieuvd/lvdthieu/CodeGen/java_data/data/deepseek.parquet", "fastparquet")
len(df[df["compile_info_filled_file_baseline_output"] == "<COMPILED_SUCCESSFULLY>"]) / len(df)
df.to_csv("check.csv")

In [8]:
import pandas as pd

df = pd.read_parquet("/home/hieuvd/lvdthieu/CodeGen/java_data/data/compile_info_deepseek_v2.parquet", "fastparquet")
# df.to_parquet("/home/hieuvd/lvdthieu/CodeGen/java_data/data/deepseek.parquet", "fastparquet")
len(df[df["compile_info_filled_file_finetune_output"] == "<COMPILED_SUCCESSFULLY>"]) / len(df)

0.9754753375585561